In [1]:
import pandas as pd
import numpy as np
# tensorflow
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, Flatten, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
Friday_WorkingHours_Afternoon_DDos            = pd.read_csv("/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
Friday_WorkingHours_Afternoon_PortScan        = pd.read_csv("/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
Friday_WorkingHours_Morning                   = pd.read_csv("/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv")
Monday_WorkingHours                           = pd.read_csv("/kaggle/input/network-intrusion-dataset/Monday-WorkingHours.pcap_ISCX.csv")
Thursday_WorkingHours_Afternoon_Infilteration = pd.read_csv("/kaggle/input/network-intrusion-dataset/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
Thursday_WorkingHours_Morning_WebAttacks      = pd.read_csv("/kaggle/input/network-intrusion-dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
Tuesday_WorkingHours                          = pd.read_csv("/kaggle/input/network-intrusion-dataset/Tuesday-WorkingHours.pcap_ISCX.csv")
Wednesday_workingHours                        = pd.read_csv("/kaggle/input/network-intrusion-dataset/Wednesday-workingHours.pcap_ISCX.csv")


In [3]:
df = pd.concat(
    [
        Friday_WorkingHours_Afternoon_DDos,
        Friday_WorkingHours_Afternoon_PortScan,
        Friday_WorkingHours_Morning,
        Monday_WorkingHours,
        Thursday_WorkingHours_Afternoon_Infilteration,
        Thursday_WorkingHours_Morning_WebAttacks,
        Tuesday_WorkingHours,
        Wednesday_workingHours
    ],
    axis=0
)
df.columns = Friday_WorkingHours_Afternoon_DDos.columns

In [4]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [5]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [6]:
df[' Label'].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed'], dtype=object)

In [7]:
df[' Label'] = df[' Label'].apply(lambda x: 'BENIGN' if x == 'BENIGN' else 'ATTACK')
df[' Label'].unique()

array(['BENIGN', 'ATTACK'], dtype=object)

In [8]:
encoder = LabelEncoder()
df[' Label']= encoder.fit_transform(df[' Label'])

In [9]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,1
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,1
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,1
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,1
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,1


In [10]:
df[' Label'].value_counts().sum

<bound method Series.sum of  Label
1    2273097
0     557646
Name: count, dtype: int64>

In [11]:
df = df.fillna(0)  # Replace NaN with 0
df = df.replace([np.inf, -np.inf], 0)

In [12]:
df.isnull().sum()

 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64

In [13]:
df=df.astype(int)

In [14]:

X = df.drop(' Label',axis=1)
y = df[' Label']




In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [16]:
# Impute missing values (replace NaNs with the mean)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Determine the number of columns (features) in your DataFrame
num_columns = df.shape[1]

# Set an appropriate value for k (less than or equal to the number of columns)
k = min(20, num_columns)  # Adjust this as needed

# Initialize SelectKBest with the scoring function
k_best = SelectKBest(score_func=f_classif, k=k)

# Fit and transform the imputed data to select the top 10 features
X_new = k_best.fit_transform(X_imputed, y)

/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [31 33 56 57 58 59 60 61] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [17]:
# Get the boolean mask of selected features
selected_features_mask = k_best.get_support()
print(selected_features_mask)



[False  True False False False False False False False False  True  True
  True  True False False False  True  True False  True False  True  True
 False False False False False False False False False False False False
 False False  True  True  True  True  True False False False False False
 False False False False  True False  True False False False False False
 False False False False False False False False False False False False
 False False  True False  True  True]


In [18]:
elected_feature_names = X.columns[selected_features_mask]

In [19]:
elected_feature_names

Index([' Flow Duration', 'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Flow IAT Std',
       ' Flow IAT Max', 'Fwd IAT Total', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', ' Average Packet Size',
       ' Avg Bwd Segment Size', 'Idle Mean', ' Idle Max', ' Idle Min'],
      dtype='object')

In [20]:
new_columns=[' Flow Duration', 'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Flow IAT Std',
       ' Flow IAT Max', 'Fwd IAT Total', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', ' Average Packet Size',
       ' Avg Bwd Segment Size', 'Idle Mean', ' Idle Max', ' Idle Min']

In [21]:
df_new=X[new_columns]

In [22]:
df_new

,Flow Duration,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Std,Flow IAT Max,Fwd IAT Total,Fwd IAT Std,Fwd IAT Max,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,Average Packet Size,Avg Bwd Segment Size,Idle Mean,Idle Max,Idle Min
0,3,0,0,0,0,0,3,3,0,3,6,6,6,0,0,9,0,0,0,0
1,109,6,6,6,0,0,109,0,0,0,6,6,6,0,0,9,6,0,0,0
2,52,6,6,6,0,0,52,0,0,0,6,6,6,0,0,9,6,0,0,0
3,34,6,6,6,0,0,34,0,0,0,6,6,6,0,0,9,6,0,0,0
4,3,0,0,0,0,0,3,3,0,3,6,6,6,0,0,9,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692698,32215,76,76,76,0,13617,30780,30832,17755,30780,28,76,41,23,548,48,76,0,0,0
692699,324,181,181,181,0,183,320,2,0,2,42,181,97,76,5796,122,181,0,0,0
692700,82,6,6,6,0,52,78,4,0,4,0,31,17,16,267,22,6,0,0,0
692701,1048635,128,128,128,0,375521,1000947,1033613,444210,1000947,32,128,53,42,1792,60,128,0,0,0


In [23]:
df_new.columns

Index([' Flow Duration', 'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Flow IAT Std',
       ' Flow IAT Max', 'Fwd IAT Total', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', ' Average Packet Size',
       ' Avg Bwd Segment Size', 'Idle Mean', ' Idle Max', ' Idle Min'],
      dtype='object')

In [24]:
df_new['label']=df[' Label']

/tmp/ipykernel_26/3066840877.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['label']=df[' Label']


In [25]:
X1=df_new.iloc[:,:-1].values
y1=df_new.iloc[:,-1].values

In [26]:
len(X1[0])

20

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=42)

In [28]:
X_test.shape, y_test.shape

((849223, 20), (849223,))

In [29]:
ann = Sequential()
ann.add(Dense(units=20,activation='sigmoid'))
ann.add(Dense(units=20,activation='sigmoid'))
ann.add(Dense(units=1,activation='sigmoid'))
ann.compile(optimizer ='adam' , loss='categorical_crossentropy' , metrics= ['accuracy'])

In [30]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [31]:
class ClassificationModel:
    def __init__(self, numerical_feature_count, num_classes):
        """
        Initializes the ClassificationModel class.
        
        :param numerical_feature_count: Number of numerical features.
        :param num_classes: Number of classes in the target variable.
        """
        self.numerical_feature_count = numerical_feature_count
        self.num_classes = num_classes
        self.model = self._build_model()

    def _build_model(self):
        """
        Builds the TensorFlow model for classification tasks.
        
        :return: Compiled TensorFlow model.
        """
        # Numerical input
        numerical_input = Input(shape=(self.numerical_feature_count,), name="numerical_input")
        x = BatchNormalization()(numerical_input)  # Normalize numerical inputs

        # Fully connected layers
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.3)(x)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.3)(x)
        x = Dense(32, activation='relu')(x)

        # Output layer
        output = Dense(self.num_classes, activation='softmax', name="output")(x)

        # Create model
        model = Model(inputs=numerical_input, outputs=output)
        
        # Compile model
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model

    def summary(self):
        """Prints the summary of the model."""
        self.model.summary()

numerical_feature_count = 20
num_classes = 2

model = ClassificationModel(numerical_feature_count, num_classes).model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 numerical_input (InputLaye  [(None, 20)]              0         
 r)                                                              
                                                                 
 batch_normalization (Batch  (None, 20)                80        
 Normalization)                                                  
                                                                 
 dense_3 (Dense)             (None, 128)               2688      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0     

In [32]:
#training the model on training set
model.fit(X_train,y_train,batch_size=32,epochs=100,callbacks=[early_stopping])

Epoch 1/100
61923/61923 [==============================] - 184s 3ms/step - loss: 0.2069 - accuracy: 0.9021
Epoch 2/100
61923/61923 [==============================] - 180s 3ms/step - loss: 0.1930 - accuracy: 0.9083
Epoch 3/100
61923/61923 [==============================] - 180s 3ms/step - loss: 0.1902 - accuracy: 0.9097
Epoch 4/100
61923/61923 [==============================] - 180s 3ms/step - loss: 0.1883 - accuracy: 0.9103
Epoch 5/100
61923/61923 [==============================] - 180s 3ms/step - loss: 0.1873 - accuracy: 0.9109
Epoch 6/100
61923/61923 [==============================] - 180s 3ms/step - loss: 0.1860 - accuracy: 0.9116
Epoch 7/100
61923/61923 [==============================] - 181s 3ms/step - loss: 0.1859 - accuracy: 0.9117
Epoch 8/100
61923/61923 [==============================] - 182s 3ms/step - loss: 0.1857 - accuracy: 0.9119
Epoch 9/100
61923/61923 [==============================] - 184s 3ms/step - loss: 0.1855 - accuracy: 0.9118
Epoch 10/100
61923/61923 [===========

In [33]:
#gives the probabibility wheather the customer will churn or not
print(model.predict([[3268,72,72,0,0,0,0,201,72,32,3268,72,72,0,0,0,0,201,72,32]]))

1/1 [==============================] - 0s 125ms/step
[[0. 1.]]


In [34]:
lr = LinearRegression()

In [35]:
X_train, X_test, y_train, y_test = train_test_split(df_new, y, test_size=0.3, random_state=42)

In [36]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (1981520, 21)
X_test shape: (849223, 21)
y_train shape: (1981520,)
y_test shape: (849223,)


In [37]:
lr.fit(X_train,y_train)

LinearRegression()

In [38]:
y_pred = lr.predict(X_test)

In [39]:
print("Mean Squared Error:",  mean_squared_error(y_test, y_pred))
print("R-squared:", r2_score(y_test, y_pred))

Mean Squared Error: 9.855604712561328e-31
R-squared: 1.0


In [40]:
model.save("model.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
